In [24]:

!pip install xgboost
!pip install polars
!pip install optuna
!pip install missingno
!pip install lightgbm
!pip install catboost


In [25]:
import numpy as np
import polars as pl
import pandas as pd
from sklearn.base import clone
from copy import deepcopy
import optuna
from scipy.optimize import minimize
import matplotlib.pyplot as plt
import missingno as msno
import re
from colorama import Fore, Style

from tqdm import tqdm
from IPython.display import clear_output

import warnings
warnings.filterwarnings('ignore')
pd.options.display.max_columns = None

import lightgbm as lgb
from catboost import CatBoostRegressor, CatBoostClassifier
from xgboost import XGBRegressor
from catboost import CatBoostRegressor
import xgboost as xgb
from sklearn.ensemble import VotingRegressor
from sklearn.model_selection import *
from sklearn.metrics import *

SEED = 5400
n_splits = 5

train = pd.read_csv("/kaggle/input/child-mind-institute-problematic-internet-use/train.csv")
test =pd.read_csv("/kaggle/input/child-mind-institute-problematic-internet-use/test.csv")


train = train.dropna(subset=['sii'])


train['PAQ_PAQ_Total'] = train['PAQ_C-PAQ_C_Total'].add(train['PAQ_A-PAQ_A_Total'],fill_value=0)
# train['PAQ_PAQ_Total'].isnull().sum()/len(train['PAQ_PAQ_Total'])
# train
train['Fitness_Endurance_Time']= train['Fitness_Endurance-Time_Mins']*60+train['Fitness_Endurance-Time_Sec']
train=train.drop('PAQ_C-PAQ_C_Total', axis=1)
train=train.drop('PAQ_A-PAQ_A_Total', axis=1)
train=train.drop('Fitness_Endurance-Time_Mins', axis=1)
train=train.drop('Fitness_Endurance-Time_Sec', axis=1)

test['PAQ_PAQ_Total'] = test['PAQ_C-PAQ_C_Total'].add(test['PAQ_A-PAQ_A_Total'],fill_value=0)
test['Fitness_Endurance_Time']= test['Fitness_Endurance-Time_Mins']*60+test['Fitness_Endurance-Time_Sec']
test=test.drop('PAQ_C-PAQ_C_Total', axis=1)
test=test.drop('PAQ_A-PAQ_A_Total', axis=1)
test=test.drop('Fitness_Endurance-Time_Mins', axis=1)
test=test.drop('Fitness_Endurance-Time_Sec', axis=1)

test_id = test['id']

train = train.drop('id',axis=1)
test = test.drop('id',axis=1)

featuresCols = ['Basic_Demos-Enroll_Season', 'Basic_Demos-Age', 'Basic_Demos-Sex',
       'CGAS-Season', 'CGAS-CGAS_Score', 'Physical-Season', 'Physical-BMI',
       'Physical-Height', 'Physical-Weight', 'Physical-Waist_Circumference',
       'Physical-Diastolic_BP', 'Physical-HeartRate', 'Physical-Systolic_BP',
       'Fitness_Endurance-Season', 'Fitness_Endurance-Max_Stage',
       'Fitness_Endurance_Time',
       'FGC-Season', 'FGC-FGC_CU', 'FGC-FGC_CU_Zone', 'FGC-FGC_GSND',
       'FGC-FGC_GSND_Zone', 'FGC-FGC_GSD', 'FGC-FGC_GSD_Zone', 'FGC-FGC_PU',
       'FGC-FGC_PU_Zone', 'FGC-FGC_SRL', 'FGC-FGC_SRL_Zone', 'FGC-FGC_SRR',
       'FGC-FGC_SRR_Zone', 'FGC-FGC_TL', 'FGC-FGC_TL_Zone', 'BIA-Season',
       'BIA-BIA_Activity_Level_num', 'BIA-BIA_BMC', 'BIA-BIA_BMI',
       'BIA-BIA_BMR', 'BIA-BIA_DEE', 'BIA-BIA_ECW', 'BIA-BIA_FFM',
       'BIA-BIA_FFMI', 'BIA-BIA_FMI', 'BIA-BIA_Fat', 'BIA-BIA_Frame_num',
       'BIA-BIA_ICW', 'BIA-BIA_LDM', 'BIA-BIA_LST', 'BIA-BIA_SMM',
       'BIA-BIA_TBW', 'PAQ_A-Season', 'PAQ_PAQ_Total', 'PAQ_C-Season',
       'SDS-Season', 'SDS-SDS_Total_Raw',
       'SDS-SDS_Total_T', 'PreInt_EduHx-Season',
       'PreInt_EduHx-computerinternet_hoursday','sii']

train = train[featuresCols]


cat_c = ['Basic_Demos-Enroll_Season','CGAS-Season','Physical-Season','Fitness_Endurance-Season','FGC-Season',
 'BIA-Season','PAQ_A-Season','PAQ_C-Season','SDS-Season','PreInt_EduHx-Season']



def update(df):
    global cat_c
    for c in cat_c : 
        df[c] = df[c].fillna('Missing')
        df[c] = df[c].astype('category')
        
    return df

def create_mapping(column, dataset):
    unique_values = dataset[column].unique()
    return {value: idx for idx, value in enumerate(unique_values)}

    
for col in cat_c:
    all_values = pd.concat([train[col], test[col]]).unique()
    mapping = {value: idx for idx, value in enumerate(all_values)}

    train[col] = train[col].replace(mapping).astype(int)
    test[col] = test[col].replace(mapping).astype(int)



In [66]:
import pandas as pd
import numpy as np
from lightgbm import LGBMRegressor
from catboost import CatBoostClassifier
from xgboost import XGBClassifier

def fill_missing_with_models(train, test, target_column, n_estimators=1000, random_state=5400):
    train = train.copy()
    test = test.copy()

    # Determine the model type
    if train[target_column].dtype == 'object' or target_column in train.select_dtypes(include=['object', 'category']).columns:
        model_type = 'classification'
    else:
        model_type = 'regression'

    # Add a flag to differentiate train and test
    train['is_train'] = 1
    test['is_train'] = 0

    # Combine train and test
    df = pd.concat([train, test], ignore_index=True)

    # Identify features excluding the target column and 'is_train'
    features_columns = [col for col in df.columns if col not in [target_column, 'is_train']]

    # Drop non-numeric columns from features for regression tasks
    if model_type == 'regression':
        numeric_features = df[features_columns].select_dtypes(include=['number']).columns.tolist()
        features_columns = numeric_features

    # Split into rows with and without missing values
    df_missing = df[df[target_column].isnull()]
    df_not_missing = df[~df[target_column].isnull()]

    if df_missing.empty or df_not_missing.empty:
        print(f"No missing data in '{target_column}' column or all data are missing.")
        return train.drop(columns=['is_train']), test.drop(columns=['is_train'])

    # Prepare training data
    X_train = df_not_missing[features_columns]
    y_train = df_not_missing[target_column]

    # Prepare model
    if model_type == 'regression':
        model = LGBMRegressor(n_estimators=n_estimators, random_state=random_state)
    else:
        model = XGBClassifier(n_estimators=n_estimators, random_state=random_state, verbose=0)  # Suppress verbose output

    # Fit the model
    model.fit(X_train, y_train)

    # Predict missing values
    X_pred = df_missing[features_columns]
    y_pred = model.predict(X_pred)

    # Fill missing values
    df.loc[df[target_column].isnull(), target_column] = y_pred

    # Split back into train and test
    train_filled = df[df['is_train'] == 1].drop(columns=['is_train'])
    test_filled = df[df['is_train'] == 0].drop(columns=['is_train'])

    return train_filled, test_filled


# List of columns to fill

# Becasue Fitness_Endurance_Time didn't get fixed well due to high range
#log
train['Fitness_Endurance_Time']=np.log(train['Fitness_Endurance_Time'])
test['Fitness_Endurance_Time']=np.log(test['Fitness_Endurance_Time'])

'''
# z standardize
train['Fitness_Endurance_Time'] = (train['Fitness_Endurance_Time'] - train['Fitness_Endurance_Time'].mean()) / train['Fitness_Endurance_Time'].std()
test['Fitness_Endurance_Time'] = (test['Fitness_Endurance_Time'] - test['Fitness_Endurance_Time'].mean()) / test['Fitness_Endurance_Time'].std()
'''


basic_cols = [
    'Basic_Demos-Age', 
    'CGAS-CGAS_Score', 'Physical-BMI', 'Physical-Height', 'Physical-Weight',
    'Physical-Waist_Circumference',
    'Physical-Diastolic_BP', 'Physical-HeartRate', 'Physical-Systolic_BP',
    'SDS-SDS_Total_Raw', 'SDS-SDS_Total_T', 'PreInt_EduHx-computerinternet_hoursday',
    'PAQ_PAQ_Total','Basic_Demos-Sex','Fitness_Endurance-Max_Stage','Fitness_Endurance_Time',
]

# Define categorical columns if necessary
cat_c1 = ['Basic_Demos-Sex','Fitness_Endurance-Max_Stage']  # Replace with a list of categorical columns, if applicable
train[cat_c1]=train[cat_c1].astype('category')
test[cat_c1]=test[cat_c1].astype('category')

# Loop through columns to fill missing values
for col in basic_cols:
    print(f"开始填补特征: {col}")
    train, test = fill_missing_with_models(train, test, col)

train['Physical-HeartRate'] = train['Physical-HeartRate'].astype(int)
test['Physical-HeartRate'] = test['Physical-HeartRate'].astype(int)

开始填补特征: Basic_Demos-Age
No missing data in 'Basic_Demos-Age' column or all data are missing.
开始填补特征: CGAS-CGAS_Score
No missing data in 'CGAS-CGAS_Score' column or all data are missing.
开始填补特征: Physical-BMI
No missing data in 'Physical-BMI' column or all data are missing.
开始填补特征: Physical-Height
No missing data in 'Physical-Height' column or all data are missing.
开始填补特征: Physical-Weight
No missing data in 'Physical-Weight' column or all data are missing.
开始填补特征: Physical-Waist_Circumference
No missing data in 'Physical-Waist_Circumference' column or all data are missing.
开始填补特征: Physical-Diastolic_BP
No missing data in 'Physical-Diastolic_BP' column or all data are missing.
开始填补特征: Physical-HeartRate
No missing data in 'Physical-HeartRate' column or all data are missing.
开始填补特征: Physical-Systolic_BP
No missing data in 'Physical-Systolic_BP' column or all data are missing.
开始填补特征: SDS-SDS_Total_Raw
No missing data in 'SDS-SDS_Total_Raw' column or all data are missing.
开始填补特征: SDS-SDS_To

In [67]:
train['BIA-Season'] = train['BIA-Season'].fillna('Other')
test['BIA-Season'] = test['BIA-Season'].fillna('Other')

# if only use BIA variables, still have missing, using Phsical- variables help
BIA = ['Basic_Demos-Age', 'Basic_Demos-Sex',
    'CGAS-CGAS_Score', 'Physical-BMI', 'Physical-Height', 'Physical-Weight',
    'Physical-Waist_Circumference','Fitness_Endurance-Max_Stage',
    'Physical-Diastolic_BP', 'Physical-HeartRate', 'Physical-Systolic_BP',
    'SDS-SDS_Total_Raw', 'SDS-SDS_Total_T', 'PreInt_EduHx-computerinternet_hoursday',
    'PAQ_PAQ_Total','Fitness_Endurance_Time',
       'BIA-Season',
       "BIA-BIA_TBW","BIA-BIA_DEE","BIA-BIA_BMC","BIA-BIA_Fat", "BIA-BIA_BMI",'BIA-BIA_Frame_num',
       'BIA-BIA_ICW', 'BIA-BIA_LDM', 'BIA-BIA_LST', 'BIA-BIA_SMM','BIA-BIA_Activity_Level_num',
      'BIA-BIA_BMR', 'BIA-BIA_ECW', 'BIA-BIA_FFM', 'BIA-BIA_FFMI', 'BIA-BIA_FMI']



for col in BIA:
    print("开始填补特征："+ col)
    train, test = fill_missing_with_models(train, test, col)


train['BIA-BIA_Frame_num'] = train['BIA-BIA_Frame_num'].astype(int)
test['BIA-BIA_Frame_num'] = test['BIA-BIA_Frame_num'].astype(int)

开始填补特征：Basic_Demos-Age
No missing data in 'Basic_Demos-Age' column or all data are missing.
开始填补特征：Basic_Demos-Sex
No missing data in 'Basic_Demos-Sex' column or all data are missing.
开始填补特征：CGAS-CGAS_Score
No missing data in 'CGAS-CGAS_Score' column or all data are missing.
开始填补特征：Physical-BMI
No missing data in 'Physical-BMI' column or all data are missing.
开始填补特征：Physical-Height
No missing data in 'Physical-Height' column or all data are missing.
开始填补特征：Physical-Weight
No missing data in 'Physical-Weight' column or all data are missing.
开始填补特征：Physical-Waist_Circumference
No missing data in 'Physical-Waist_Circumference' column or all data are missing.
开始填补特征：Fitness_Endurance-Max_Stage
No missing data in 'Fitness_Endurance-Max_Stage' column or all data are missing.
开始填补特征：Physical-Diastolic_BP
No missing data in 'Physical-Diastolic_BP' column or all data are missing.
开始填补特征：Physical-HeartRate
No missing data in 'Physical-HeartRate' column or all data are missing.
开始填补特征：Physical-Sy

In [68]:
import pandas as pd
import numpy as np
from lightgbm import LGBMRegressor
from catboost import CatBoostClassifier
from xgboost import XGBClassifier

def fill_missing_with_models(train, test, target_column, n_estimators=1000, random_state=5400):
    train = train.copy()
    test = test.copy()

    # Determine the model type
    if train[target_column].dtype == 'object' or target_column in train.select_dtypes(include=['object', 'category']).columns:
        model_type = 'classification'
    else:
        model_type = 'regression'

    # Add a flag to differentiate train and test
    train['is_train'] = 1
    test['is_train'] = 0

    # Combine train and test
    df = pd.concat([train, test], ignore_index=True)

    # Identify features excluding the target column and 'is_train'
    features_columns = [col for col in df.columns if col not in [target_column, 'is_train']]

    # Drop non-numeric columns from features for regression tasks
    if model_type == 'regression':
        numeric_features = df[features_columns].select_dtypes(include=['number']).columns.tolist()
        features_columns = numeric_features

    # Split into rows with and without missing values
    df_missing = df[df[target_column].isnull()]
    df_not_missing = df[~df[target_column].isnull()]

    if df_missing.empty or df_not_missing.empty:
        print(f"No missing data in '{target_column}' column or all data are missing.")
        return train.drop(columns=['is_train']), test.drop(columns=['is_train'])

    # Prepare training data
    X_train = df_not_missing[features_columns]
    y_train = df_not_missing[target_column]

    # Prepare model
    if model_type == 'regression':
        model = XGBRegressor(n_estimators=n_estimators, random_state=random_state)
    else:
        model = XGBClassifier(n_estimators=n_estimators, random_state=random_state, verbose=0)  # Suppress verbose output

    # Fit the model
    model.fit(X_train, y_train)

    # Predict missing values
    X_pred = df_missing[features_columns]
    y_pred = model.predict(X_pred)

    # Fill missing values
    df.loc[df[target_column].isnull(), target_column] = y_pred

    # Split back into train and test
    train_filled = df[df['is_train'] == 1].drop(columns=['is_train'])
    test_filled = df[df['is_train'] == 0].drop(columns=['is_train'])

    return train_filled, test_filled


FGC_cols = ['Basic_Demos-Age', 
    'Physical-BMI', 'Physical-Height', 'Physical-Weight',
    'Physical-Waist_Circumference',
    'Physical-Diastolic_BP', 'Physical-Systolic_BP',
    'Fitness_Endurance-Max_Stage','Fitness_Endurance_Time',
  'FGC-FGC_CU',
  'FGC-FGC_CU_Zone',
  'FGC-FGC_GSND',
  'FGC-FGC_GSND_Zone',
  'FGC-FGC_GSD',
  'FGC-FGC_GSD_Zone',
  'FGC-FGC_PU',
  'FGC-FGC_PU_Zone',
  'FGC-FGC_SRL',
  'FGC-FGC_SRL_Zone',
  'FGC-FGC_SRR',
  'FGC-FGC_SRR_Zone',
  'FGC-FGC_TL',
  'FGC-FGC_TL_Zone'
]



for col in FGC_cols:
    print("开始填补特征："+ col)
    train, test = fill_missing_with_models(train, test, col)

train['FGC-FGC_CU_Zone']=train['FGC-FGC_CU_Zone'].astype(int)
train['FGC-FGC_GSND_Zone']=train['FGC-FGC_GSND_Zone'].astype(int)
train['FGC-FGC_GSD_Zone']=train['FGC-FGC_GSD_Zone'].astype(int)
train['FGC-FGC_SRL_Zone']=train['FGC-FGC_SRL_Zone'].astype(int)
train['FGC-FGC_SRR_Zone']=train['FGC-FGC_SRR_Zone'].astype(int)
train['FGC-FGC_TL_Zone']=train['FGC-FGC_TL_Zone'].astype(int)


test['FGC-FGC_CU_Zone']=test['FGC-FGC_CU_Zone'].astype(int)
test['FGC-FGC_GSND_Zone']=test['FGC-FGC_GSND_Zone'].astype(int)
test['FGC-FGC_GSD_Zone']=test['FGC-FGC_GSD_Zone'].astype(int)
test['FGC-FGC_SRL_Zone']=test['FGC-FGC_SRL_Zone'].astype(int)
test['FGC-FGC_SRR_Zone']=test['FGC-FGC_SRR_Zone'].astype(int)
test['FGC-FGC_TL_Zone']=test['FGC-FGC_TL_Zone'].astype(int)

开始填补特征：Basic_Demos-Age
No missing data in 'Basic_Demos-Age' column or all data are missing.
开始填补特征：Physical-BMI
No missing data in 'Physical-BMI' column or all data are missing.
开始填补特征：Physical-Height
No missing data in 'Physical-Height' column or all data are missing.
开始填补特征：Physical-Weight
No missing data in 'Physical-Weight' column or all data are missing.
开始填补特征：Physical-Waist_Circumference
No missing data in 'Physical-Waist_Circumference' column or all data are missing.
开始填补特征：Physical-Diastolic_BP
No missing data in 'Physical-Diastolic_BP' column or all data are missing.
开始填补特征：Physical-Systolic_BP
No missing data in 'Physical-Systolic_BP' column or all data are missing.
开始填补特征：Fitness_Endurance-Max_Stage
No missing data in 'Fitness_Endurance-Max_Stage' column or all data are missing.
开始填补特征：Fitness_Endurance_Time
No missing data in 'Fitness_Endurance_Time' column or all data are missing.
开始填补特征：FGC-FGC_CU
No missing data in 'FGC-FGC_CU' column or all data are missing.
开始填补特征：FGC

In [69]:
train = update(train)
test = update(test)

# interaction feature
def create_interaction_features(df, feature_pairs):
    global cat_c
    for feature1, feature2 in feature_pairs:
        if feature1 not in cat_c or feature2 not in cat_c:
            print("feature1: " + feature1 + ", feature2: " + feature2)
            new_feature_name = f"{feature1}x{feature2}"
            df[new_feature_name] = df[feature1] * df[feature2]
    return df

def create_division_features(df, feature_pairs):
    global cat_c
    for feature1, feature2 in feature_pairs:
        if feature1 not in cat_c or feature2 not in cat_c:
            print(f"feature1: {feature1}, feature2: {feature2}")
            
            # Create A/B feature, handle division by zero or NaN
            new_feature_name1 = f"{feature1}div{feature2}"
            df[new_feature_name1] = df[feature1] / df[feature2]
            
            # Mask NaN or zero values
            df[new_feature_name1] = df[new_feature_name1].mask((df[feature1] == 0) | (df[feature2] == 0) | 
                                                               df[feature1].isna() | df[feature2].isna())
            
            # Create B/A feature, handle similarly
            new_feature_name2 = f"{feature2}div{feature1}"
            df[new_feature_name2] = df[feature2] / df[feature1]
            df[new_feature_name2] = df[new_feature_name2].mask((df[feature1] == 0) | (df[feature2] == 0) | 
                                                               df[feature1].isna() | df[feature2].isna())

    return df

feature_pairs = [
    ('PreInt_EduHx-computerinternet_hoursday', 'Basic_Demos-Age'),
    ('Basic_Demos-Age', 'SDS-SDS_Total_T'),
    ('FGC-FGC_SRR_Zone', 'SDS-SDS_Total_T'),
    ('BIA-BIA_BMC', 'Physical-HeartRate'),
    #('Fitness_Endurance-Season', 'Physical-Waist_Circumference'),
    ('BIA-BIA_Fat', 'Physical-BMI'),
    ('PreInt_EduHx-Season', 'Fitness_Endurance-Season'),
    ('SDS-SDS_Total_T', 'Physical-Systolic_BP'),
    ('BIA-BIA_Activity_Level_num', 'FGC-FGC_PU_Zone')
]

train = create_interaction_features(train,feature_pairs)
test = create_interaction_features(test,feature_pairs)
train = create_division_features(train,feature_pairs)
test = create_division_features(test,feature_pairs)

feature1: PreInt_EduHx-computerinternet_hoursday, feature2: Basic_Demos-Age
feature1: Basic_Demos-Age, feature2: SDS-SDS_Total_T
feature1: FGC-FGC_SRR_Zone, feature2: SDS-SDS_Total_T
feature1: BIA-BIA_BMC, feature2: Physical-HeartRate
feature1: BIA-BIA_Fat, feature2: Physical-BMI
feature1: SDS-SDS_Total_T, feature2: Physical-Systolic_BP
feature1: BIA-BIA_Activity_Level_num, feature2: FGC-FGC_PU_Zone
feature1: PreInt_EduHx-computerinternet_hoursday, feature2: Basic_Demos-Age
feature1: Basic_Demos-Age, feature2: SDS-SDS_Total_T
feature1: FGC-FGC_SRR_Zone, feature2: SDS-SDS_Total_T
feature1: BIA-BIA_BMC, feature2: Physical-HeartRate
feature1: BIA-BIA_Fat, feature2: Physical-BMI
feature1: SDS-SDS_Total_T, feature2: Physical-Systolic_BP
feature1: BIA-BIA_Activity_Level_num, feature2: FGC-FGC_PU_Zone
feature1: PreInt_EduHx-computerinternet_hoursday, feature2: Basic_Demos-Age
feature1: Basic_Demos-Age, feature2: SDS-SDS_Total_T
feature1: FGC-FGC_SRR_Zone, feature2: SDS-SDS_Total_T
feature1: B

In [70]:
missing_percent_train = train.isnull().mean() * 100
missing_percent_train

Basic_Demos-Enroll_Season                        0.000000
Basic_Demos-Age                                  0.000000
Basic_Demos-Sex                                  0.000000
CGAS-Season                                      0.000000
CGAS-CGAS_Score                                  0.000000
                                                  ...    
Physical-BMIdivBIA-BIA_Fat                       0.255848
SDS-SDS_Total_TdivPhysical-Systolic_BP           0.000000
Physical-Systolic_BPdivSDS-SDS_Total_T           0.000000
BIA-BIA_Activity_Level_numdivFGC-FGC_PU_Zone    44.736842
FGC-FGC_PU_ZonedivBIA-BIA_Activity_Level_num    44.736842
Length: 78, dtype: float64

In [71]:
def quadratic_weighted_kappa(y_true, y_pred):
    return cohen_kappa_score(y_true, y_pred, weights='quadratic')

def threshold_Rounder(oof_non_rounded, thresholds):
    return np.where(oof_non_rounded < thresholds[0], 0,
                    np.where(oof_non_rounded < thresholds[1], 1,
                             np.where(oof_non_rounded < thresholds[2], 2, 3)))

def evaluate_predictions(thresholds, y_true, oof_non_rounded):
    rounded_p = threshold_Rounder(oof_non_rounded, thresholds)
    return -quadratic_weighted_kappa(y_true, rounded_p)

def TrainML(model_class, test_data):
    
    X = train.drop(['sii'], axis=1)
    y = train['sii']
    test_data = test_data.drop(['sii'], axis=1)
    SKF = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=SEED)
    
    train_S = []
    test_S = []
    
    oof_non_rounded = np.zeros(len(y), dtype=float) 
    oof_rounded = np.zeros(len(y), dtype=int) 
    test_preds = np.zeros((len(test_data), n_splits))

    for fold, (train_idx, test_idx) in enumerate(tqdm(SKF.split(X, y), desc="Training Folds", total=n_splits)):
        X_train, X_val = X.iloc[train_idx], X.iloc[test_idx]
        y_train, y_val = y.iloc[train_idx], y.iloc[test_idx]

        model = clone(model_class)
        model.fit(X_train, y_train)

        y_train_pred = model.predict(X_train)
        y_val_pred = model.predict(X_val)

        oof_non_rounded[test_idx] = y_val_pred
        y_val_pred_rounded = y_val_pred.round(0).astype(int)
        oof_rounded[test_idx] = y_val_pred_rounded

        train_kappa = quadratic_weighted_kappa(y_train, y_train_pred.round(0).astype(int))
        val_kappa = quadratic_weighted_kappa(y_val, y_val_pred_rounded)

        train_S.append(train_kappa)
        test_S.append(val_kappa)
        
        test_preds[:, fold] = model.predict(test_data)
        
        print(f"Fold {fold+1} - Train QWK: {train_kappa:.4f}, Validation QWK: {val_kappa:.4f}")
        clear_output(wait=True)

    print(f"Mean Train QWK --> {np.mean(train_S):.4f}")
    print(f"Mean Validation QWK ---> {np.mean(test_S):.4f}")

    KappaOPtimizer = minimize(evaluate_predictions,
                              x0=[0.5, 1.5, 2.5], args=(y, oof_non_rounded), 
                              method='Nelder-Mead') # Nelder-Mead | # Powell
    assert KappaOPtimizer.success, "Optimization did not converge."
    
    oof_tuned = threshold_Rounder(oof_non_rounded, KappaOPtimizer.x)
    tKappa = quadratic_weighted_kappa(y, oof_tuned)

    print(f"----> || Optimized QWK SCORE :: {Fore.CYAN}{Style.BRIGHT} {tKappa:.3f}{Style.RESET_ALL}")

    tpm = test_preds.mean(axis=1)
    tpTuned = threshold_Rounder(tpm, KappaOPtimizer.x)
    
    submission = pd.DataFrame({
        'id': test_id,
        'sii': tpTuned
    })


    return submission, tKappa


from sklearn.preprocessing import LabelEncoder

# Function to preprocess data for LightGBM
def preprocess_data(df):
    # Identify non-numeric columns
    non_numeric_cols = df.select_dtypes(include=['object']).columns
    
    # Apply label encoding to non-numeric columns
    label_encoders = {}
    for col in non_numeric_cols:
        le = LabelEncoder()
        df[col] = le.fit_transform(df[col].astype(str))
        label_encoders[col] = le
    
    return df, label_encoders


# Apply preprocessing to both train and test data
train_preprocessed, train_encoders = preprocess_data(train)
test_preprocessed, _ = preprocess_data(test)




best_params_lgbm = {
    'learning_rate': 0.046,
    'max_depth': 12,
    'num_leaves': 478,
    'min_data_in_leaf': 13,
    'feature_fraction': 0.893,
    'bagging_fraction': 0.784,
    'bagging_freq': 4,
    'lambda_l1': 10,
    'lambda_l2': 0.01
}

best_params_xgb = {
'subsample': 0.8,
'reg_lambda': 100, 
'reg_alpha': 0, 
'n_estimators': 500,
'min_child_weight': 5,
'max_depth': 3, 
'learning_rate': 0.01,
'gamma': 0, 
'colsample_bytree': 1.0
}

best_params_catboost = {
    'random_strength': 1,
 'learning_rate': 0.01,
 'l2_leaf_reg': 9, 
 'iterations': 500, 
 'depth': 4, 
 'border_count': 64,
 'bagging_temperature': 0.2}

best_params_rf =  {'n_estimators': 400,
                   'min_samples_split': 5, 
                   'min_samples_leaf': 4, 
                   'max_features': 'sqrt',
                   'max_depth': 10, 
                   'bootstrap': True}


import numpy as np
import pandas as pd
from lightgbm import LGBMRegressor
from xgboost import XGBRegressor
from sklearn.experimental import enable_hist_gradient_boosting  # Required for HistGradientBoosting
from sklearn.ensemble import HistGradientBoostingRegressor
from catboost import CatBoostRegressor
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import cohen_kappa_score
from sklearn.ensemble import VotingRegressor
from scipy.optimize import minimize


from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import LabelEncoder

# Encode categorical features using LabelEncoder
cate_c = [
    'Basic_Demos-Enroll_Season', 'CGAS-Season', 'Physical-Season',
    'Fitness_Endurance-Season', 'FGC-Season', 'BIA-Season',
    'PAQ_A-Season', 'PAQ_C-Season', 'SDS-Season',
    'PreInt_EduHx-Season', 'Basic_Demos-Sex'
]


In [72]:
Light = lgb.LGBMRegressor(**best_params_lgbm, random_state=SEED, verbose=-1)
Submission_LGBM, k_lgbm = TrainML(Light, test_preprocessed)

XGBoost = xgb.XGBRegressor(**best_params_xgb, random_state=SEED,enable_categorical=True)
Submission_XGB, k_xgb = TrainML(XGBoost, test)

cate_c = ['Basic_Demos-Enroll_Season',
 'CGAS-Season',
 'Physical-Season',
 'Fitness_Endurance-Season',
 'FGC-Season',
 'BIA-Season',
 'PAQ_A-Season',
 'PAQ_C-Season',
 'SDS-Season',
 'PreInt_EduHx-Season',
 'Basic_Demos-Sex'
        ]

CatBoost = CatBoostRegressor(**best_params_catboost, random_state=SEED, verbose=0,cat_features=cate_c)
Submission_CatBoost , k_cat= TrainML(CatBoost, test)

Training Folds: 100%|██████████| 5/5 [00:03<00:00,  1.53it/s]

Mean Train QWK --> 0.6344
Mean Validation QWK ---> 0.3725


----> || Optimized QWK SCORE ::  0.456


In [93]:
import pandas as pd
from scipy.stats import mode

merged_predictions = Submission_LGBM.merge(Submission_XGB, on="id", suffixes=("_lgbm", "_xgb"))
merged_predictions = merged_predictions.merge(
    Submission_CatBoost.rename(columns={"sii": "sii_catboost"}), on="id"
)

merged_predictions



,id,sii_lgbm,sii_xgb,sii_catboost
0,00008ff9,0,0,0
1,000fd460,0,0,0
2,00105258,0,0,0
3,00115b9f,0,0,0
4,0016bb22,1,2,2
5,001f3379,0,0,0
6,0038ba98,0,1,1
7,0068a485,1,1,1
8,0069fbed,2,2,2
9,0083e397,2,2,2


In [94]:
import pandas as pd
from collections import Counter

# Custom function to calculate the mode robustly
def calculate_mode(values):
    counter = Counter(values)
    return counter.most_common(1)[0][0]  # Get the most common value

# Apply the function row-wise to calculate the mode
merged_predictions["sii"] = merged_predictions[["sii_lgbm", "sii_xgb", "sii_catboost"]].apply(
    lambda row: calculate_mode(row), axis=1
)

merged_predictions = merged_predictions.drop(["sii_lgbm", "sii_xgb", "sii_catboost"], axis=1)
merged_predictions

merged_predictions.to_csv('submission.csv', index=False)

In [95]:
merged_predictions

,id,sii
0,00008ff9,0
1,000fd460,0
2,00105258,0
3,00115b9f,0
4,0016bb22,2
5,001f3379,0
6,0038ba98,1
7,0068a485,1
8,0069fbed,2
9,0083e397,2
